# Lab 2.2.3: Object Detection Demo

**Module:** 2.2 - Computer Vision  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand the difference between classification, detection, and segmentation
- [ ] Know the evolution from R-CNN to YOLO
- [ ] Use YOLOv8 for real-time object detection
- [ ] Benchmark detection speed on DGX Spark

---

## 📚 Prerequisites

- Completed: Tasks 7.1-7.2
- Knowledge of: CNNs, transfer learning

---

## 🌍 Real-World Context

**Object detection is everywhere:**

- 🚗 **Autonomous vehicles**: Detecting pedestrians, other cars, traffic signs
- 📷 **Security cameras**: Identifying suspicious activities
- 🏭 **Manufacturing**: Quality control and defect detection
- 🛒 **Retail**: Inventory management and checkout automation
- 🏥 **Medical imaging**: Detecting tumors, lesions, anomalies

---

## 🧒 ELI5: What is Object Detection?

> **Imagine you're playing "I Spy" but with a twist...** 🔍
>
> - **Classification**: "I spy something... it's a CAT!" (What is in the image?)
> - **Detection**: "I spy a CAT at (x=100, y=200)!" (What AND where?)
> - **Segmentation**: "I spy a CAT and here's its exact silhouette!" (Pixel-perfect boundary)
>
> Object detection answers: **What objects are in the image, and WHERE are they?**

### Visual Comparison

```
Classification:        Detection:              Segmentation:
┌──────────────┐       ┌──────────────┐        ┌──────────────┐
│              │       │ ┌────┐       │        │    ▓▓▓       │
│    🐱        │       │ │ 🐱 │       │        │   ▓▓▓▓▓      │
│              │       │ └────┘       │        │  ▓▓▓▓▓▓▓     │
│    🐕        │       │    ┌────┐    │        │      ████    │
│              │       │    │ 🐕 │    │        │     ██████   │
└──────────────┘       └────┴────┴────┘        └──────────────┘
Output: "cat, dog"     Output: boxes +         Output: pixel
                       labels + confidence     masks for each
```

---

## Part 1: Evolution of Object Detection

### The Two Families of Detectors

```
TWO-STAGE DETECTORS:                    ONE-STAGE DETECTORS:
┌───────────────────────────┐           ┌───────────────────────────┐
│                           │           │                           │
│  Step 1: Propose regions  │           │  Single pass: detect      │
│  "Where might objects be?"│           │  everything at once!      │
│            │              │           │                           │
│            ▼              │           │  Input → CNN → Boxes      │
│  Step 2: Classify regions │           │                           │
│  "What is in each region?"│           │                           │
│                           │           │                           │
└───────────────────────────┘           └───────────────────────────┘

Examples:                               Examples:
- R-CNN (2014)                          - YOLO (2016)
- Fast R-CNN (2015)                     - SSD (2016)
- Faster R-CNN (2016)                   - RetinaNet (2017)

Pros: More accurate                     Pros: Much faster!
Cons: Slower (two steps)                Cons: Slightly less accurate
```

### YOLO: "You Only Look Once"

The key insight of YOLO: **Detection can be a single regression problem!**

```
Image → CNN → Grid predictions → Filter by confidence → Final boxes
              ┌─────────────┐
              │ x, y, w, h  │  (box coordinates)
              │ confidence  │  (objectness score)
              │ class probs │  (80 COCO classes)
              └─────────────┘
```

In [ ]:
# Setup
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
from pathlib import Path
import urllib.request
from typing import List, Tuple, Dict

# Install ultralytics for YOLOv8
# Note: For DGX Spark ARM64, pre-install in NGC container for best results
# See README.md for Quick Start instructions

try:
    from ultralytics import YOLO
    print("✅ Ultralytics YOLOv8 available")
except ImportError:
    print("⚠️ ultralytics not found. Installing...")
    print("   💡 TIP: For DGX Spark ARM64, this may take a moment to compile native extensions.")
    print("   For faster startup, pre-install in your NGC container: pip install ultralytics")
    !pip install ultralytics -q
    from ultralytics import YOLO
    print("✅ ultralytics installed successfully!")

# Check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\n🖥️  Device: {device}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

---

## Part 2: YOLOv8 Model Zoo

YOLOv8 comes in different sizes for different use cases:

In [ ]:
# YOLOv8 model variants
yolov8_models = {
    'yolov8n': {'params': 3.2, 'mAP': 37.3, 'speed': 'Fastest'},
    'yolov8s': {'params': 11.2, 'mAP': 44.9, 'speed': 'Fast'},
    'yolov8m': {'params': 25.9, 'mAP': 50.2, 'speed': 'Medium'},
    'yolov8l': {'params': 43.7, 'mAP': 52.9, 'speed': 'Slow'},
    'yolov8x': {'params': 68.2, 'mAP': 53.9, 'speed': 'Slowest'},
}

print("📊 YOLOv8 Model Variants:")
print("="*60)
print(f"{'Model':<12} {'Parameters':>12} {'mAP@0.5':>12} {'Speed':>12}")
print("-"*60)
for name, info in yolov8_models.items():
    print(f"{name:<12} {info['params']:>10.1f}M {info['mAP']:>11.1f}% {info['speed']:>12}")
print("\n💡 Tip: Start with yolov8s for a good balance!")

In [ ]:
# Download sample images for detection
def download_sample_images():
    """Download sample images for detection demos."""
    data_dir = Path('../data/detection_samples')
    data_dir.mkdir(parents=True, exist_ok=True)
    
    # Sample images from COCO dataset
    urls = {
        'street.jpg': 'https://ultralytics.com/images/bus.jpg',
        'zidane.jpg': 'https://ultralytics.com/images/zidane.jpg',
    }
    
    for filename, url in urls.items():
        filepath = data_dir / filename
        if not filepath.exists():
            print(f"Downloading {filename}...")
            urllib.request.urlretrieve(url, filepath)
    
    return data_dir

sample_dir = download_sample_images()
print(f"✅ Sample images ready in {sample_dir}")

---

## Part 3: Running YOLOv8 Inference

Let's detect objects in sample images!

In [ ]:
# Load YOLOv8 model (downloads automatically)
print("Loading YOLOv8s model...")
model = YOLO('yolov8s.pt')  # 's' for small, good balance

print(f"✅ Model loaded!")
print(f"   Model type: {model.task}")
print(f"   Classes: {len(model.names)} (COCO dataset)")

In [ ]:
# Run detection on sample image
image_path = sample_dir / 'street.jpg'

# Inference
results = model(str(image_path), verbose=False)

# Access first result (we only passed one image)
result = results[0]

print(f"📊 Detection Results:")
print(f"   Image size: {result.orig_shape}")
print(f"   Objects found: {len(result.boxes)}")
print(f"   Inference time: {result.speed['inference']:.1f}ms")

In [ ]:
def visualize_detections(result, figsize=(12, 8)):
    """
    Visualize detection results with bounding boxes.
    """
    # Get annotated image
    annotated = result.plot()
    
    # Convert BGR to RGB
    annotated = annotated[:, :, ::-1]
    
    fig, axes = plt.subplots(1, 2, figsize=figsize)
    
    # Original image
    axes[0].imshow(result.orig_img[:, :, ::-1])
    axes[0].set_title('Original Image', fontsize=12)
    axes[0].axis('off')
    
    # Detections
    axes[1].imshow(annotated)
    axes[1].set_title(f'Detections ({len(result.boxes)} objects)', fontsize=12)
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print detection details
    print("\n🔍 Detection Details:")
    print("-"*50)
    for i, box in enumerate(result.boxes):
        cls_id = int(box.cls[0])
        cls_name = result.names[cls_id]
        conf = float(box.conf[0])
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        print(f"   {i+1}. {cls_name}: {conf:.1%} @ [{x1:.0f}, {y1:.0f}, {x2:.0f}, {y2:.0f}]")

visualize_detections(result)

In [ ]:
# Try another image
image_path2 = sample_dir / 'zidane.jpg'
results2 = model(str(image_path2), verbose=False)
visualize_detections(results2[0])

---

## Part 4: Understanding Detection Outputs

Let's dive deeper into what YOLO outputs.

In [ ]:
def analyze_detections(result) -> Dict:
    """
    Analyze detection results in detail.
    
    Returns:
        Dictionary with detection statistics
    """
    boxes = result.boxes
    
    # Extract data
    class_ids = boxes.cls.cpu().numpy().astype(int)
    confidences = boxes.conf.cpu().numpy()
    xyxy = boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
    
    # Class distribution
    class_counts = {}
    for cls_id in class_ids:
        cls_name = result.names[cls_id]
        class_counts[cls_name] = class_counts.get(cls_name, 0) + 1
    
    # Box sizes
    widths = xyxy[:, 2] - xyxy[:, 0]
    heights = xyxy[:, 3] - xyxy[:, 1]
    areas = widths * heights
    
    stats = {
        'num_detections': len(boxes),
        'class_distribution': class_counts,
        'confidence_range': (confidences.min(), confidences.max()),
        'avg_confidence': confidences.mean(),
        'box_area_range': (areas.min(), areas.max()),
        'avg_box_area': areas.mean(),
    }
    
    return stats

stats = analyze_detections(result)

print("📊 Detection Analysis:")
print("="*50)
print(f"Total detections: {stats['num_detections']}")
print(f"\nClass distribution:")
for cls_name, count in stats['class_distribution'].items():
    print(f"   {cls_name}: {count}")
print(f"\nConfidence: {stats['confidence_range'][0]:.1%} - {stats['confidence_range'][1]:.1%}")
print(f"Average confidence: {stats['avg_confidence']:.1%}")
print(f"\nBox area: {stats['box_area_range'][0]:.0f} - {stats['box_area_range'][1]:.0f} pixels²")

### 🧒 ELI5: Non-Maximum Suppression (NMS)

> **Imagine you're looking for faces in a crowd photo...**
>
> The model might draw 10 slightly overlapping boxes around the same face because it's not sure exactly where the face boundaries are.
>
> **NMS (Non-Maximum Suppression)** cleans this up:
> 1. Keep the most confident box
> 2. Remove any overlapping boxes (above IoU threshold)
> 3. Repeat for remaining boxes
>
> Result: One clean box per object!

In [ ]:
# Demonstrate confidence threshold effect
def compare_confidence_thresholds(image_path, thresholds=[0.1, 0.25, 0.5, 0.75]):
    """
    Show how confidence threshold affects detection.
    """
    fig, axes = plt.subplots(1, len(thresholds), figsize=(4*len(thresholds), 4))
    
    for ax, conf_thresh in zip(axes, thresholds):
        results = model(str(image_path), conf=conf_thresh, verbose=False)
        annotated = results[0].plot()
        
        ax.imshow(annotated[:, :, ::-1])
        ax.set_title(f'Confidence > {conf_thresh}\n({len(results[0].boxes)} objects)', fontsize=10)
        ax.axis('off')
    
    plt.suptitle('🎯 Effect of Confidence Threshold', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

compare_confidence_thresholds(sample_dir / 'street.jpg')

---

## Part 5: DGX Spark Benchmarks

Let's benchmark YOLOv8 performance on DGX Spark!

In [ ]:
# Benchmark configuration
BENCHMARK_WARMUP = 5    # Number of warmup iterations
BENCHMARK_RUNS = 20     # Number of timed iterations

def benchmark_yolo(
    model_size: str = 'yolov8s',
    image_sizes: List[int] = [320, 640, 1280],
    num_warmup: int = BENCHMARK_WARMUP,
    num_runs: int = BENCHMARK_RUNS
) -> Dict:
    """
    Benchmark YOLO model at different image sizes.
    
    Args:
        model_size: YOLO model variant (yolov8n, yolov8s, yolov8m, etc.)
        image_sizes: List of image sizes to test
        num_warmup: Number of warmup iterations before timing
        num_runs: Number of timed iterations for averaging
    
    Returns:
        Dictionary of benchmarks by image size
    """
    print(f"🏎️ Benchmarking {model_size}...")
    model = YOLO(f'{model_size}.pt')
    
    # Generate random image
    results = {}
    
    for img_size in image_sizes:
        print(f"\n   Testing {img_size}×{img_size}...")
        
        # Create dummy image
        dummy_img = np.random.randint(0, 255, (img_size, img_size, 3), dtype=np.uint8)
        
        # Warmup
        for _ in range(num_warmup):
            model(dummy_img, imgsz=img_size, verbose=False)
        
        # Benchmark
        torch.cuda.synchronize() if torch.cuda.is_available() else None
        times = []
        
        for _ in range(num_runs):
            start = time.perf_counter()
            model(dummy_img, imgsz=img_size, verbose=False)
            torch.cuda.synchronize() if torch.cuda.is_available() else None
            times.append(time.perf_counter() - start)
        
        avg_time = np.mean(times) * 1000  # Convert to ms
        fps = 1000 / avg_time
        
        results[img_size] = {
            'avg_time_ms': avg_time,
            'fps': fps,
            'times': times
        }
        
        print(f"      Avg time: {avg_time:.1f}ms, FPS: {fps:.1f}")
    
    return results

benchmarks = benchmark_yolo('yolov8s')

In [ ]:
# Compare all model sizes
def benchmark_all_models(
    models: List[str] = ['yolov8n', 'yolov8s', 'yolov8m'],
    img_size: int = 640,
    num_warmup: int = BENCHMARK_WARMUP,
    num_runs: int = BENCHMARK_RUNS
) -> Dict:
    """
    Benchmark multiple YOLO variants.
    
    Args:
        models: List of YOLO model variants to benchmark
        img_size: Image size for benchmarking
        num_warmup: Number of warmup iterations before timing
        num_runs: Number of timed iterations for averaging
    
    Returns:
        Dictionary of benchmark results per model
    """
    results = {}
    dummy_img = np.random.randint(0, 255, (img_size, img_size, 3), dtype=np.uint8)
    
    for model_name in models:
        print(f"\n🏎️ Benchmarking {model_name}...")
        model = YOLO(f'{model_name}.pt')
        
        # Warmup
        for _ in range(num_warmup):
            model(dummy_img, imgsz=img_size, verbose=False)
        
        # Benchmark
        torch.cuda.synchronize() if torch.cuda.is_available() else None
        times = []
        
        for _ in range(num_runs):
            start = time.perf_counter()
            model(dummy_img, imgsz=img_size, verbose=False)
            torch.cuda.synchronize() if torch.cuda.is_available() else None
            times.append(time.perf_counter() - start)
        
        avg_time = np.mean(times) * 1000
        results[model_name] = {
            'avg_time_ms': avg_time,
            'fps': 1000 / avg_time,
            'params': yolov8_models[model_name]['params'],
            'mAP': yolov8_models[model_name]['mAP']
        }
        
        print(f"   ✅ {avg_time:.1f}ms ({1000/avg_time:.1f} FPS)")
        
        # Cleanup
        del model
        torch.cuda.empty_cache()
    
    return results

all_benchmarks = benchmark_all_models()

In [ ]:
# Visualize benchmark results
def plot_benchmarks(benchmarks: Dict):
    """Create benchmark visualization."""
    models = list(benchmarks.keys())
    fps = [benchmarks[m]['fps'] for m in models]
    mAP = [benchmarks[m]['mAP'] for m in models]
    params = [benchmarks[m]['params'] for m in models]
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(models)))
    
    # FPS comparison
    bars = axes[0].bar(models, fps, color=colors)
    axes[0].set_ylabel('Frames Per Second')
    axes[0].set_title('⚡ Speed (Higher is Better)')
    for bar, f in zip(bars, fps):
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                    f'{f:.0f}', ha='center', va='bottom', fontweight='bold')
    
    # mAP comparison
    bars = axes[1].bar(models, mAP, color=colors)
    axes[1].set_ylabel('mAP@0.5 (%)')
    axes[1].set_title('🎯 Accuracy (Higher is Better)')
    for bar, m in zip(bars, mAP):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                    f'{m:.1f}', ha='center', va='bottom', fontweight='bold')
    
    # Speed vs Accuracy scatter
    scatter = axes[2].scatter(fps, mAP, c=params, s=[p*20 for p in params],
                              cmap='viridis', alpha=0.7)
    for m, f, a in zip(models, fps, mAP):
        axes[2].annotate(m, (f, a), textcoords='offset points', 
                        xytext=(5, 5), fontsize=9)
    axes[2].set_xlabel('FPS')
    axes[2].set_ylabel('mAP@0.5 (%)')
    axes[2].set_title('📊 Speed vs Accuracy Trade-off')
    cbar = plt.colorbar(scatter, ax=axes[2])
    cbar.set_label('Parameters (M)')
    
    plt.suptitle('🏎️ YOLOv8 Model Comparison on DGX Spark', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

plot_benchmarks(all_benchmarks)

In [ ]:
# Summary table
print("\n" + "="*70)
print("📊 DGX SPARK BENCHMARK SUMMARY")
print("="*70)
print(f"{'Model':<12} {'Parameters':>12} {'mAP@0.5':>12} {'Time (ms)':>12} {'FPS':>12}")
print("-"*70)
for name, data in all_benchmarks.items():
    print(f"{name:<12} {data['params']:>10.1f}M {data['mAP']:>11.1f}% {data['avg_time_ms']:>11.1f} {data['fps']:>11.1f}")
print("="*70)

# Recommendations
fastest = max(all_benchmarks, key=lambda x: all_benchmarks[x]['fps'])
most_accurate = max(all_benchmarks, key=lambda x: all_benchmarks[x]['mAP'])

print(f"\n💡 Recommendations:")
print(f"   - For real-time video (30+ FPS): Use {fastest}")
print(f"   - For best accuracy: Use {most_accurate}")
print(f"   - For balanced performance: Use yolov8s")

---

## Part 6: Batch Processing

Process multiple images efficiently.

In [ ]:
def process_batch(model, image_paths: List[str], batch_size: int = 4):
    """
    Process images in batches for efficiency.
    """
    all_results = []
    
    for i in range(0, len(image_paths), batch_size):
        batch = image_paths[i:i+batch_size]
        results = model(batch, verbose=False)
        all_results.extend(results)
    
    return all_results

# Demo with both sample images
image_paths = [str(sample_dir / 'street.jpg'), str(sample_dir / 'zidane.jpg')]
model = YOLO('yolov8s.pt')

start = time.time()
batch_results = process_batch(model, image_paths * 5)  # Process 10 images
batch_time = time.time() - start

print(f"\n⚡ Batch Processing:")
print(f"   Images processed: {len(batch_results)}")
print(f"   Total time: {batch_time:.2f}s")
print(f"   Throughput: {len(batch_results)/batch_time:.1f} images/sec")

---

## Part 7: Custom Detection Filtering

Sometimes you only care about specific classes.

In [ ]:
# Print all COCO classes
print("📋 All COCO Classes:")
print("="*60)
for i, name in model.names.items():
    print(f"{i:>3}: {name:<20}", end='')
    if (i + 1) % 4 == 0:
        print()

In [ ]:
def detect_specific_classes(image_path: str, classes: List[str]):
    """
    Detect only specific classes.
    
    Args:
        image_path: Path to image
        classes: List of class names to detect
    """
    # Get class indices
    class_to_idx = {v: k for k, v in model.names.items()}
    class_indices = [class_to_idx[c] for c in classes if c in class_to_idx]
    
    print(f"🔍 Looking for: {classes}")
    print(f"   Class indices: {class_indices}")
    
    # Run detection with class filter
    results = model(image_path, classes=class_indices, verbose=False)
    
    # Visualize
    annotated = results[0].plot()
    plt.figure(figsize=(10, 8))
    plt.imshow(annotated[:, :, ::-1])
    plt.title(f"Detecting: {', '.join(classes)} ({len(results[0].boxes)} found)")
    plt.axis('off')
    plt.show()

# Detect only people
detect_specific_classes(str(sample_dir / 'street.jpg'), ['person', 'car', 'bus'])

---

## ✋ Try It Yourself

1. Download an image from the internet and run detection on it
2. Filter for specific objects relevant to your use case
3. Compare YOLOv8n vs YOLOv8x on the same image

<details>
<summary>💡 Hint</summary>

```python
# Download image
url = "your_image_url"
urllib.request.urlretrieve(url, "my_image.jpg")

# Detect
results = model("my_image.jpg")
```

</details>

In [ ]:
# YOUR CODE HERE



---

## ⚠️ Common Mistakes

### Mistake 1: Wrong input format

```python
# ❌ Wrong: Normalized tensor input
img_tensor = transforms.ToTensor()(img)  # Values 0-1
results = model(img_tensor)  # May give wrong results!

# ✅ Right: Use raw image (PIL, numpy, or path)
results = model("image.jpg")  # Path
results = model(np.array(img))  # NumPy array (0-255)
```
**Why:** YOLOv8 handles preprocessing internally.

### Mistake 2: Ignoring confidence threshold

```python
# ❌ Wrong: Too low threshold (many false positives)
results = model(img, conf=0.1)

# ✅ Right: Appropriate threshold for your use case
results = model(img, conf=0.5)  # Higher precision
```
**Why:** Low threshold gives many false positives. Tune for your needs.

### Mistake 3: Not using GPU

```python
# ❌ Wrong: CPU inference (slow!)
model = YOLO('yolov8s.pt')
# No explicit device set, may default to CPU

# ✅ Right: Ensure GPU inference
model = YOLO('yolov8s.pt')
model.to('cuda')  # Or check model.device
```
**Why:** GPU is 10-100× faster for inference.

---

## 🎉 Checkpoint

You've learned:
- ✅ Difference between classification, detection, and segmentation
- ✅ Evolution from R-CNN to YOLO
- ✅ Using YOLOv8 for object detection
- ✅ Benchmarking detection speed on DGX Spark
- ✅ Filtering detections by confidence and class

---

## 🚀 Challenge (Optional)

**Create a simple video processing pipeline:**

1. Download a short video or use webcam
2. Process frame-by-frame with YOLOv8
3. Count unique objects over time
4. Save annotated video

<details>
<summary>💡 Starting Code</summary>

```python
import cv2

cap = cv2.VideoCapture('video.mp4')
model = YOLO('yolov8n.pt')  # Use nano for speed

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    results = model(frame, verbose=False)
    annotated = results[0].plot()
    
    cv2.imshow('Detection', annotated)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
```

</details>

In [ ]:
# YOUR CHALLENGE CODE HERE



---

## 📖 Further Reading

- [Ultralytics YOLOv8 Documentation](https://docs.ultralytics.com/)
- [YOLOv8 GitHub](https://github.com/ultralytics/ultralytics)
- [Original YOLO Paper](https://arxiv.org/abs/1506.02640)
- [COCO Dataset](https://cocodataset.org/)

---

## 🧹 Cleanup

In [ ]:
# Clear GPU memory
import gc

del model
torch.cuda.empty_cache()
gc.collect()

print("✅ Cleanup complete!")
if torch.cuda.is_available():
    print(f"💾 GPU Memory Free: {torch.cuda.mem_get_info()[0] / 1e9:.1f} GB")